In [ ]:
# SHORT-TERM INVENTORY MODEL (STIM)

In [23]:
# ---------------------------
# PRE-PROCESSING CODE
# ---------------------------

import pandas as pd
import glob
import os
from sklearn.preprocessing import LabelEncoder

print('Fetching CSV files...')

# Define folder path and read all CSVs inside
folder = r'C:\Users\Dree\Desktop\ITM_project\ACTUAL CODING\CSV'
csv_files = glob.glob(os.path.join(folder, '*.csv'))
csv_list = [pd.read_csv(file) for file in csv_files]

# Combine all CSVs into one DataFrame
agg_csv = pd.concat(csv_list, ignore_index=True)

# Filter and clean relevant columns
agg_csv = agg_csv[[
    'order_date_time',
    'category',
    'barcode',
    'product',
    'pc_quantity',
    'price',
    'total_product_price'
]].copy()

# Extract date only from datetime
agg_csv['order_date'] = pd.to_datetime(agg_csv['order_date_time']).dt.floor('D')

# Group by product and date
agg_csv = (
    agg_csv.drop(columns=['order_date_time'])
    .groupby(['barcode', 'product', 'price', 'category', 'order_date'], as_index=False)
    [['pc_quantity', 'total_product_price']].sum()
)

# ---------------------------
# Feature Engineering Section
# ---------------------------

# Day of the week & weekend flag
agg_csv['day_of_week'] = agg_csv['order_date'].dt.dayofweek
agg_csv['if_weekend'] = agg_csv['day_of_week'] >= 5

# Extract month
agg_csv['month'] = agg_csv['order_date'].dt.month

# Sort by barcode and order_date
agg_csv = agg_csv.sort_values(by=['barcode', 'order_date'])

# Sales from the last order
agg_csv['sales_last_order'] = agg_csv.groupby('barcode')['total_product_price'].shift(1)

# Rolling 3-order sum and mean (excluding current row)
agg_csv['sales_last_3orders'] = (
    agg_csv.groupby('barcode')['total_product_price']
    .shift(1)
    .rolling(window=3, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

agg_csv['rolling_mean_3orders'] = (
    agg_csv.groupby('barcode')['total_product_price']
    .shift(1)
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# Difference from previous order
agg_csv['sales_diff_prev_order'] = agg_csv['total_product_price'] - agg_csv['sales_last_order']

# Encode category labels for ML
label_encoder = LabelEncoder()
agg_csv['category_label'] = label_encoder.fit_transform(agg_csv['category'])

# Replace NaNs with 0
agg_csv = agg_csv.fillna(0)

print('Preprocessing completed!')

Fetching CSV files...
Preprocessing completed!


In [24]:
# ---------------------------
# MODEL CODE
# ---------------------------

import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

print('Fetching data...')

# -------------------
# Feature & Target Setup
# -------------------
X = agg_csv[[
    'price',
    'category_label',
    'day_of_week',
    'if_weekend',
    'month',
    'sales_last_order',
    'sales_last_3orders',
    'rolling_mean_3orders',
    'sales_diff_prev_order'
]]

y = agg_csv['pc_quantity']

# -------------------
# Train/Test Split
# -------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

# -------------------
# Base Model Training
# -------------------
base_model = RandomForestRegressor(random_state=77)
base_model.fit(X_train, y_train)

# -------------------
# Base Model Evaluation
# -------------------
y_pred = base_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Base Model Performance:")
print(f"MAE  = {mae:.2f}")
print(f"RMSE = {rmse:.2f}")
print(f"R²   = {r2:.4f}")

# -------------------
# Hyperparameter Tuning
# -------------------
param_grid = {
    'n_estimators': [100, 300, 500, 700],
    'max_features': [None],
    'max_depth': [10, 20, 30], 
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print('Tuning model with GridSearchCV...')

grid = GridSearchCV(
    RandomForestRegressor(random_state=77),
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid.fit(X_train, y_train)

print("Best parameters found:", grid.best_params_)
print("Best score (negative MSE):", grid.best_score_)

# -------------------
# Save Best Model
# -------------------
best_model = grid.best_estimator_
joblib.dump(best_model, 'SKU-prediction-model.joblib')

print("Model saved as 'SKU-prediction-model.joblib'.")

Fetching data...
Base Model Performance:
MAE  = 29.98
RMSE = 195.51
R²   = 0.7999
Tuning model with GridSearchCV...
Best parameters found: {'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Best score (negative MSE): -12815.441993270244
Model saved as 'SKU-prediction-model.joblib'.


In [2]:
import pandas as pd
import numpy as np
import os
import glob
import math
import joblib
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

print('Fetching CSV files...')

# === 1. LOAD & AGGREGATE CSV FILES ===
folder = r'C:\Users\Dree\Desktop\ITM_project\ACTUAL CODING\FOR_PREDICTION'
csv_files = glob.glob(os.path.join(folder, '*.csv'))
csv_list = [pd.read_csv(file) for file in csv_files]
agg_csv = pd.concat(csv_list, ignore_index=True)

# === 2. CLEAN & GROUP RAW DATA ===
agg_csv = agg_csv[[
    'order_date_time', 'category', 'barcode', 'product',
    'pc_quantity', 'price', 'total_product_price'
]].copy()

agg_csv['order_date'] = pd.to_datetime(agg_csv['order_date_time']).dt.floor('D')
agg_csv = (
    agg_csv.drop(columns='order_date_time')
           .groupby(['barcode', 'product', 'price', 'category', 'order_date'], as_index=False)
           .agg({'pc_quantity': 'sum', 'total_product_price': 'sum'})
)

# === 3. FEATURE ENGINEERING ===
agg_csv['day_of_week'] = agg_csv['order_date'].dt.dayofweek
agg_csv['if_weekend'] = agg_csv['day_of_week'] >= 5
agg_csv['month'] = agg_csv['order_date'].dt.month

agg_csv = agg_csv.sort_values(by=['barcode', 'order_date'])

agg_csv['sales_last_order'] = agg_csv.groupby('barcode')['total_product_price'].shift(1)

agg_csv['sales_last_3orders'] = (
    agg_csv.groupby('barcode')['total_product_price']
           .shift(1)
           .rolling(window=3, min_periods=1)
           .sum()
           .reset_index(level=0, drop=True)
)

agg_csv['rolling_mean_3orders'] = (
    agg_csv.groupby('barcode')['total_product_price']
           .shift(1)
           .rolling(window=3, min_periods=1)
           .mean()
           .reset_index(level=0, drop=True)
)

agg_csv['sales_diff_prev_order'] = (
    agg_csv['total_product_price'] - agg_csv['sales_last_order']
)

agg_csv['category_label'] = LabelEncoder().fit_transform(agg_csv['category'])

agg_csv = agg_csv.fillna(0)

# === 4. BUILD FINAL INPUT FOR PREDICTION ===
predict_csv = agg_csv[[
    'barcode', 'product', 'price', 'category_label', 'order_date',
    'day_of_week', 'if_weekend', 'month',
    'sales_last_order', 'sales_last_3orders',
    'rolling_mean_3orders', 'sales_diff_prev_order'
]].copy()

predict_csv = (
    predict_csv.loc[predict_csv.groupby('barcode')['order_date'].idxmax()]
                 .reset_index(drop=True)
)

# === 5. LOAD MODEL & RUN PREDICTIONS ===
print('Loading model...')
model = joblib.load('SKU-prediction-model.joblib')
print('Model loaded!')

forecast_days = 7
future_preds = []

for _, row in predict_csv.iterrows():
    try:
        sku_preds = []
        prediction_date = row['order_date']
        prev_sales = row['sales_last_order']

        sales_history = [
            row['sales_last_order'],
            row['sales_last_3orders'] - row['sales_last_order'],
            row['rolling_mean_3orders']
        ]

        for _ in range(forecast_days):
            prediction_date += timedelta(days=1)
            day_of_week = prediction_date.dayofweek
            is_weekend = day_of_week >= 5
            month = prediction_date.month

            X_input = pd.DataFrame([{
                'price': row['price'],
                'category_label': row['category_label'],
                'day_of_week': day_of_week,
                'if_weekend': is_weekend,
                'month': month,
                'sales_last_order': prev_sales,
                'sales_last_3orders': sum(sales_history),
                'rolling_mean_3orders': np.mean(sales_history),
                'sales_diff_prev_order': (
                    prev_sales - sales_history[-2] if len(sales_history) > 1 else 0
                ),
            }])

            predicted_sales = max(0, math.ceil(model.predict(X_input)[0]))

            sku_preds.append({
                'barcode': row['barcode'],
                'product': row['product'],
                'prediction_date': prediction_date,
                'predicted_inventory': predicted_sales
            })

            sales_history = sales_history[-2:] + [predicted_sales]
            prev_sales = predicted_sales

        future_preds.extend(sku_preds)
        print(f"Finished SKU: {row['barcode']}")

    except Exception as e:
        print(f"Error on SKU {row['barcode']}: {e}")

# === 6. OUTPUT FINAL PREDICTIONS ===
future_df = pd.DataFrame(future_preds)

output = (
    future_df.groupby(['barcode', 'product'], as_index=False)[['predicted_inventory']]
             .sum()
             .copy()
)

# === 7. EXPORT TO CSV ===
output_path = r'C:\Users\Dree\Desktop\ITM_project\ACTUAL CODING\predicted_inventory.csv'
output.to_csv(output_path, index=False)
print(f"Predictions saved to: {output_path}")

Fetching CSV files...
Loading model...
Model loaded!
Finished SKU: 501120020053
Finished SKU: 501120020663
Finished SKU: 501120020976
Finished SKU: 501120030003
Finished SKU: 501120030106
Finished SKU: 501120030146
Finished SKU: 501120030153
Finished SKU: 501120030154
Finished SKU: 501120030155
Finished SKU: 501120030158
Finished SKU: 501120030159
Finished SKU: 501120030161
Finished SKU: 501120030162
Finished SKU: 501120030164
Finished SKU: 501120030165
Finished SKU: 501120030167
Finished SKU: 501120030168
Finished SKU: 501120030212
Finished SKU: 501120030234
Finished SKU: 501120030237
Finished SKU: 501120030310
Finished SKU: 501120030311
Finished SKU: 501120030312
Finished SKU: 501317010010
Finished SKU: 501317010011
Finished SKU: 501317020018
Finished SKU: 501317020026
Finished SKU: 501317020027
Finished SKU: 501318020044
Finished SKU: 501318020045
Finished SKU: 501318020046
Finished SKU: 501318020047
Finished SKU: 501318020049
Finished SKU: 501318020050
Finished SKU: 501318020057
Fi

In [7]:
# FOR PREVIEWING
# ****simulated predictions
future_df.head(14)

,barcode,product,prediction_date,predicted_inventory
0,501120020053,0053 STAR CHK 'N CHSE NUGGETS 150G X26,2025-07-16,6
1,501120020053,0053 STAR CHK 'N CHSE NUGGETS 150G X26,2025-07-17,1
2,501120020053,0053 STAR CHK 'N CHSE NUGGETS 150G X26,2025-07-18,6
3,501120020053,0053 STAR CHK 'N CHSE NUGGETS 150G X26,2025-07-19,2
4,501120020053,0053 STAR CHK 'N CHSE NUGGETS 150G X26,2025-07-20,16
...,...,...,...,...
9,501120020663,"0663 STAR, CLASSIC BURGER, 228G X 12.",2025-07-13,18
10,501120020663,"0663 STAR, CLASSIC BURGER, 228G X 12.",2025-07-14,1
11,501120020663,"0663 STAR, CLASSIC BURGER, 228G X 12.",2025-07-15,1
12,501120020663,"0663 STAR, CLASSIC BURGER, 228G X 12.",2025-07-16,1


In [4]:
# FOR PREVIEWING
pd.set_option('display.max_rows', 6)
output

,barcode,product,predicted_inventory
0,501120020053,0053 STAR CHK 'N CHSE NUGGETS 150G X26,38
1,501120020663,"0663 STAR, CLASSIC BURGER, 228G X 12.",26
2,501120020976,"0976 TJ, CD, JBO, CHEESY PIZZA, 1KGX12, WM.",132
...,...,...,...
187,5020170873544,"3544 SM COFFEE ORIG, SF, 7GX10X30, TIPID PACK",101
188,5020170874321,SM COFFEE 3IN1 SF STRONG SUP10S 9GX10X30,51
189,5020170898441,"SMC SFREE ORIG, DOUBLE TIPID, 7GX20X15.",125
